# Project: Wine

## Идеи
* Опять же, посмотреть облако слов, которыми описывают вина, сделать словарик для тех, кто хочет блеснуть в приличном обществе. 
* всего 20 уникльных тестеров, можно посомтреть, кто лучше предсказывает цену
* связь цены и оценок. Интересно, будет ли расти variance с ростом цены (типа, с дешевыми винами всё понятно, а вот дорогие более противоречивые) 
* description облако слов по вообще любым срезаам: по тестерам, по сортаам, регионам.
* цены по регионам

In [1]:
# main requirements

import pandas as pd
import numpy as np

### A. Working with ready datasets

In [2]:
data1 = pd.read_csv('winemag-data_first150k.csv')
data2 = pd.read_csv('winemag-data-130k-v2.csv')
data = [data1, data2]

In [3]:
for d in data:
    print(d.columns)
    display(d.head())

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [4]:
data2.taster_name.unique()

array(['Kerin O’Keefe', 'Roger Voss', 'Paul Gregutt',
       'Alexander Peartree', 'Michael Schachner', 'Anna Lee C. Iijima',
       'Virginie Boone', 'Matt Kettmann', nan, 'Sean P. Sullivan',
       'Jim Gordon', 'Joe Czerwinski', 'Anne Krebiehl\xa0MW',
       'Lauren Buzzeo', 'Mike DeSimone', 'Jeff Jenssen',
       'Susan Kostrzewa', 'Carrie Dykes', 'Fiona Adams',
       'Christina Pickard'], dtype=object)

### B. Manually parsing vivino.com

### 1. Beautiful Soup

We are trying to drill down to see individual wine properties in the explore page. The HTML tree classes of the original page include:
* `body class="inner-page"`
* `div class="wrap"`
* `div id="explore-page-app"`
* `div class="explorerPage__explorePage--26aGH layout__outer--S05yQ"`
* `div class="layout__inner--3JC-x"`
* `div class="explorerPage__columns--1TTaK"`
* `div class="explorerPage__results--3wqLw"`
* `div class="explorerCard__explorerCard--3Q7_0 explorerPageResults__explorerCard--3q6Qe"`
* etc

Eventually, we need the following element: `"anchor__anchor--2QZvA"` visible in order to get the link to the page of each wine

In [5]:
import requests
# from bs4 import BeautifulSoup


# headers = {
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     "Host": "www.vivino.com",
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
#     "Accept-Language": "en-gb",
#     "Accept-Encoding": "gzip, deflate, br",
#     "Connection": "keep-alive"    
# }

# response = requests.get("https://www.vivino.com/explore?e=eJwNyb0KgCAYBdC3ubNB611aWtsj4stMBH9CzertaznLCZkdgotUCPKwVwr65ThA_0w4_22a8fIe9mCT7EwVj7QxS3XRllWayWINEndTNO46L-w-zyUdWg==", headers=headers)
# content = response.content

# parser = BeautifulSoup(content, 'html.parser')
# body = parser.body
# # print(body)
# wine_titles = body.find_all(class_="vintageTitle__wine--U7t9G")
# for title in wine_titles:
#     type(title.text)
# # type(wine_titles)
# wine_titles

#### Observation on Beautiful Soup

Apparently, simple BeautifulSoup does not work since vivino.com is using JavaScript to generate dynamic web pages, therefore, individual elements of the wine list can not be parsed without JavaScript. 

The deepest element of HTML page that can be achieved with Beautiful soup is the container with id `"explore-page-app"`.

### 2. Selenium web driver

First, I considered using PhantomJS to generate web pages with datascript, see details [here](https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python).
However, after some attempts it appears that PhantomJS has been depricated, and developers of Selenium suggest using headless versions of Chrome or Firefox instead 
(see details [here](https://stackoverflow.com/questions/50416538/python-phantomjs-says-i-am-not-using-headless))


In [6]:
# from selenium import webdriver

In [7]:
# driver = webdriver.PhantomJS("/Users/sveta/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs")
# response_selenium = driver.get("https://www.vivino.com/explore?e=eJwNyb0KgCAYBdC3ubNB611aWtsj4stMBH9CzertaznLCZkdgotUCPKwVwr65ThA_0w4_22a8fIe9mCT7EwVj7QxS3XRllWayWINEndTNO46L-w-zyUdWg==")
# response_selenium
# a_elements = response_selenium.find_elements_by_tag_name("a")
# for el in a_elements:
#     print(el)

Below is the code required to initialize a web driver.

Basically, everything is done on the WebDriver instance object. The `find_element(By)` or `find_element_by_id` methods return another object type, the WebElement.
See documentation [here](https://www.selenium.dev/documentation/en/getting_started_with_webdriver/locating_elements/).

In [8]:
# Initialize web driver

# def initialize_chrome_driver(long_screen=False):
#     options = webdriver.ChromeOptions()
#     options.add_argument('--headless')
#     options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15')
#     options.add_argument("start-maximized")
#     if long_screen:
#         options.add_argument("--window-size=1920x10800")
#     else:
#         options.add_argument("--window-size=1920x1080")
#     # options.add_argument("disable-infobars")
#     # options.add_argument("--no-sandbox")
#     # options.add_argument("--disable-extensions")
#     # options.add_argument("--disable-dev-shm-usage")
#     # options.add_argument('--lang=en')
#     # options.add_argument('--incognito')

#     browser = webdriver.Chrome("/Users/sveta/Documents/Data analysis/Vivino-project/lib/chromedriver.uu", options=options)
#     return browser

### Parsing the wine list (no scroll)

In [9]:
# script to check functionality of google.com
# browser.get('http://www.google.com/xhtml')
# res = browser.find_element_by_id("SIvCob")

In order to see whether the explore page is loaded correctly, the screenshot is saved to the folder. 

After several attempts to load the webpage I get the response that my IP has been temporarily blocked for exceeding bulk request limits. Therefore, I added some properties to the Web Driver, such as setting a **User agent** that does not reveal headless nature of Chrome

In [10]:
# get the screenshot of the explore page

# browser = initialize_chrome_driver()
# test_explore_page = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1MTBQS660dXdSSwYSAWoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-MSy1KLE9FS1fNuU1OJktfKS6FhbQwDu-xpj"
# browser.get(test_explore_page)
# # res = browser.find_element_by_class_name("inner-page")
# browser.get_screenshot_as_file('test_screenshot.png')
# # browser.text

Once the page is loaded correctly, we are trying to get the individual wine properties.

Those can be found within the class `"anchor__anchor--2QZvA"`
The property of each wine has the link that leads to the individual web page of that wine. 

We are saving each link to a Pyhon list.
However, the page contains some other links unnecessary for our study (such as wine-regions, wine-sountries, etc). Those are specifically excluded. 

In [11]:
# def get_list_no_scroll(browser, page, class_name="anchor__anchor--2QZvA"):
#     browser.get(page)
#     results_list = browser.find_elements_by_class_name(class_name)
#     wine_pages_list = []
#     for el in results_list:
#         cur_link = el.get_property("href")
#         if cur_link.startswith('https://www.vivino.com/wine-countries/')\
#         or cur_link.startswith('https://www.vivino.com/wine-regions/')\
#         or cur_link.startswith('https://www.vivino.com/redirect/')\
#         or cur_link.startswith('https://instagram')\
#         or cur_link.startswith('https://facebook')\
#         or cur_link.startswith('https://twitter'):
#             continue
#         else:
#             wine_pages_list.append(cur_link)
#     return wine_pages_list

In [12]:
# browser = initialize_chrome_driver(long_screen=True)
# wine_pages_list_v2 = get_list_no_scroll(browser, test_explore_page)
# len(wine_pages_list_v2)

In [13]:
# This is a temporary solution to get the first 25 links, since sometimes the page is not loading fast enough and does not gain any useful links, so we use results that we already had before
# wine_pages_list_v2 = wine_pages_list
# len(wine_pages_list_v2)

We can see that even with JavaScript-enabled web driver, the wines are not all loaded simultaneously, and the search results in just a few elements. Therefore, we need to implement scrolling. 

### Parsing a wine list (with JavaScript scrolling)

It appears that we need a scroller to get the whole list of wines, since they are not all loaded simultaneously.

First, to implement a scroller, we need find an element that is located in the bottom of the page, and run a script until such element becomes visible.

We are trying to find an element with a class `"addWidgetLink__addWidgetLink--aPZ_V"` to indicate the page bottom, and see if the scrolling works

In [14]:
# # scroll until the end of a specific wine page 

# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException

# def scroll_until_page_bottom(browser, page, page_end_class="addWidgetLink__addWidgetLink--aPZ_V"):
#     browser.get(page)
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     try:
#         WebDriverWait(browser, 50).until(EC.visibility_of_element_located((By.CLASS_NAME, page_end_class)))
#     except TimeoutException:
#         print("exception raised")

In [15]:
# trying to extract more wines 

# # red wines with ratings of 4.5+ and price above 400 
# explore_link_1 = "https://www.vivino.com/explore?e=eJzLLbI10TNVy83MszUxMFDLTawA08mVtu5OaslAIkCtwNZQLT3NtiyxKDO1JDFHLT_JtiixJDMvvTg-sSy1KDE9VS3fNiW1OFmtvCQ61tYQADLIGy0="
# # desert wines with ratings of 4.5+ and price above 25 
# explore_link_2 = "https://www.vivino.com/explore?e=eJwNxbEKgCAUBdC_eWNYGE13aWltj4iXmQipoWL197mc4yJk05OzHl2NX0ghSH2YRlKVmW60ZE4UjlZnvijsiJytN2njoiMbTQGHToqevKwYfiK_GwY="

# # scroll_until_page_bottom(browser, explore_link_1)
# # results_list_1 = browser.find_elements_by_class_name("anchor__anchor--2QZvA")

# scroll_until_page_bottom(browser, explore_link_2)
# results_list_2 = browser.find_elements_by_class_name("anchor__anchor--2QZvA")

# wine_count_1 = len(results_list_1)
# wine_count_2 = len(results_list_2)
# print(wine_count_1)
# print(wine_count_2)

In [16]:
# browser.get_screenshot_as_file('scroll_screen.png')

Scrolling until certain element is located does not seem to work properly. 
Therefore, the next try is to scroll a page down certain number of times, and wait a bit after each scroll (to allow the page to load more wines). This solution is ispired by [this article](https://dev.to/mr_h/python-selenium-infinite-scrolling-3o12)

Therefore, we write a function that scrolls down with a certain sleep timer (2 sec) after each scroll, loads the web elements that have a certain class (`"anchor__anchor--2QZvA"`), and returns a list with such elements. 

In order to limit the desired number of elements, we can either manually restrict the length of the resulting list (using a variable `total_wine_num`, or target the overall number of existing wines, as can be seen in the top of the explorer page.

Also, since the page loads only a few new elements with each scroll, in order to increase efficiency, we check web elements every 10 scrolls (therefore, the iteration counter is required).

In [17]:
import time 

# def scroll_load_scroll(driver, page, timeout=2, class_name="anchor__anchor--2QZvA"):
    
#     # Open the explore page
#     driver.get(page)
    
#     # extract the total number of wines with a given search criteria
# #     time.sleep(timeout)
# #     total_wine_string = browser.find_element_by_class_name("querySummary__querySummary--39WP2").text
# #     total_wine_num = int(total_wine_string.split()[1])

#     total_wine_num = 50
    
#     results_list = []
#     count_iter = 0

#     while len(results_list) < total_wine_num:
        
# #         browser.get_screenshot_as_file('scroll_screen_' + str(count_iter) +'.png')
# #         print("gotcha!")

#         if count_iter % 10 == 0:

#             timepoint_1 = time.time()

#             all_results = browser.find_elements_by_class_name(class_name)

#             timepoint_2 = time.time()
#             print("time to parse elements: " + str(timepoint_2 - timepoint_1))

#             # get the link and check whether it meets the required criteria in order to be included to the list 
#             for el in all_results:
#                 cur_link = el.get_property("href")
#                 if cur_link.startswith('https://www.vivino.com/wine-countries/')\
#                 or cur_link.startswith('https://www.vivino.com/wine-regions/')\
#                 or cur_link.startswith('https://www.vivino.com/redirect/')\
#                 or cur_link.startswith('https://instagram')\
#                 or cur_link.startswith('https://facebook')\
#                 or cur_link.startswith('https://twitter')\
#                 or cur_link in results_list:
#                     continue
#                 else:
#                     results_list.append(cur_link)
        
#         timepoint_3 = time.time()
#         print("time to update list of links: " + str(timepoint_3 - timepoint_2))
        
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # Wait to load page
#         time.sleep(timeout)
        
#         timepoint_4 = time.time()
#         print("time to scroll and wait: " + str(timepoint_4 - timepoint_3))
        
#         count_iter += 1
        
#     print("list has {} elements".format(len(results_list)))
#     return results_list

The above solution `scroll_load_scroll` that loads after each scroll (or each 10 scrolls) seems pretty slow, so, alternatively `scroll_and_load` function takes a given number of scrolls as an argument, and performs this number of scrolls before loading the page.

In [18]:
# def scroll_and_load(driver, page, timeout=1, scrolls=60, class_name="anchor__anchor--2QZvA"):
#     timepoint_0 = time.time()
#     results_list = []
#     driver.get(page)
#     timepoint_1 = time.time()
#     for i in range (scrolls):
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(timeout)
#     timepoint_2 = time.time()
# #     print("time to scroll and wait: {} s.".format(timepoint_2 - timepoint_1))
#     loaded_elements = browser.find_elements_by_class_name(class_name)
#     timepoint_3 = time.time()
# #     print("time to load elements: {} s.".format(timepoint_3 - timepoint_2))
#     for el in loaded_elements:
#         cur_link = el.get_property("href")
#         if cur_link.startswith('https://www.vivino.com/wine-countries/')\
#         or cur_link.startswith('https://www.vivino.com/wine-regions/')\
#         or cur_link.startswith('https://www.vivino.com/redirect/')\
#         or cur_link.startswith('https://instagram')\
#         or cur_link.startswith('https://facebook')\
#         or cur_link.startswith('https://twitter')\
#         or cur_link in results_list:
#             continue
#         else:
#             results_list.append(cur_link)
#     timepoint_4 = time.time()
# #     print("time to extract the list of links: {} s.".format(timepoint_4 - timepoint_3))
#     print("total time elapsed: {} s.".format(timepoint_4 - timepoint_0))
#     print("list has {} elements".format(len(results_list)))
#     print("average time per element is {} s".format((timepoint_4 - timepoint_0)/len(results_list)))
#     return results_list

In [19]:
# browser = initialize_chrome_driver(long_screen=False)

In [20]:
# some other links for testing

# # all fortified wines with a rating above 4.5 (the overal result should be 423)
# explore_link_3 = "https://www.vivino.com/explore?e=eJwNxL0KgCAYBdC3uWNY2HiXltb2iPgyEyE1TPp5-zrDCZm6ahF8pEKQh1opmJd9B_M34GANt_GS7G2RHWlhluKjO2e5bBZnkbja0-Au48RGfyshGv4="
# # all red wines, sorted by rating
# explore_link_4 = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1MTBQS660dXdSSwYSAWoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-MSy1KLE9FS1fNuU1OJktfKS6FhbQwDu-xpj"
# # all red wines, sorted by popularity
# explore_link_5 = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1MTBQS660dXdSSwYSAWoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-OT80rwStXzblNTiZLXykuhYW0MAulcZsQ=="

In [21]:
# list_with_scroll = scroll_and_load(browser, explore_link_5)

In [22]:
# list_with_scroll[-1]

The above strategy (60 scrolls, each with 1s timeout) gained 1425 elements. We should experiment with a number of scrolls and/or the value of timeout to see if it helps to increase the speed of loading of new elements. Experiments performed below. Since the timing and the list size differ in different experiments, one should look to the average loading time per element to see whether a strategy is efficient or not. 

In [23]:
# browser = initialize_chrome_driver()
# list_with_scroll = scroll_and_load(browser, explore_link_5, scrolls=100)

In [24]:
# browser = initialize_chrome_driver()
# list_with_scroll = scroll_and_load(browser, explore_link_5, timeout=0.5)

In [25]:
# browser = initialize_chrome_driver()
# list_with_scroll = scroll_and_load(browser, explore_link_5, timeout=0.5, scrolls=100)

We can see that timeout of 0.5s and 100 scrolls gains relatively high speed of loading and achievs relatively more results. We will use the strategy of 0.5s and 200 scrolls further on, and apply it to various wine groups (red vs. white, and also divided by country).

To use this approach we feed individual web pages per each country and wine type to our program (sorted inthe order of descending popularity), and run a web crawler through each of them. The results are saved in a list. 

In [26]:
# argentina_red = "https://www.vivino.com/explore?e=eJwNirsOgCAMAP-mMyauXVxc3Y0xtSIhETClPvh7u9wNd0mwgxQzOkj0Ye8ccMNxADZMcFkNBz4k0SudUDYU0phDXbncWaHg7ivDq_NiK7dqJvkBsTQc7g=="
# australia_red = "https://www.vivino.com/explore?e=eJwNijsKgDAQBW_z6gi229jY2ovIumoImETyUXN7t5kpZnyiDt4FMvD8UW8MpNE4QBQTbq32pIeTOwpfiBslLi7YvEqsoSDSfmTBW-ZFV2lZzfUHsTcc8Q=="
# austria_red = "https://www.vivino.com/explore?e=eJwNijsOgCAQBW_zakxst7GxtTfGrCsSEgED64fbSzNTzIRMHYKPZBD4o94YSKVxgDRMuFp1Bz2cvVU-kTbKrD66skq6oyLRbovg1Xlpq9TSzPoDsTYc8A=="
# chile_red = "https://www.vivino.com/explore?e=eJwNijsKgDAQBW_z6gi229jY2otIXGMI5CPJ-ru928wUM6lShxQyGST7Um8M-KNxACsmnFr9QbetwYmNKBtVKyH7tnK5sqDQ7hrjkXnRlb-m5vgDsTIc6g=="
# france_red = "https://www.vivino.com/explore?e=eJwNirsOgCAMAP-mMyauXVxc3Y0xtQIhETClvv7eLnfDXRbsIKeCDjK92DsH_OE4ABsmOK3GgDdJ8koH1A2FNJXYVq5XUai4-8bw6LzYyl8zB_kBsT4c8w=="
# germany_red = "https://www.vivino.com/explore?e=eJwNijsOgCAQBW_zakxst7GxtTfG4IqERMDA-uH2bjNTzMRCHWJIZBDtR70x4EbjAFZMuLT6gx5bghN7Im9UrITk68r5ToJMu6uMV-ZFV25VvbsfsS0c5A=="
# italy_red = "https://www.vivino.com/explore?e=eJwNirsOgCAMAP-mMyauXVxc3Y0xtSJpImCgPvh7u9wNd7FgB1ESOoj0Ye8ccMNxADZMcFkNBz5UxCudkDcspJJCXTnfSSHj7ivDq_NiK7dqFv0BsUYc-A=="
# portugal_red = "https://www.vivino.com/explore?e=eJwNijsKgDAQBW_z6gi229jY2otIXGMImA_J-ru928wUM7FShxgSGUT7Um8M-KNxACsmFK3-oNvW4MSeyBtVKyH5tnK-kiDT7hrjkXnRlb-mLvIDsVQc_w=="
# spain_red = "https://www.vivino.com/explore?e=eJwNirsOgCAMAP-mMyauXVxc3Y0xtSIhETC0Pvh7u9wNd6liBylmdJDow9454IbjAGyY4LIaDnyoRq90QtmwksYcZOVyZ4WCuxeGV-fFVm5i9vIDsT0c8w=="
# usa_red = "https://www.vivino.com/explore?e=eJwNijEOgCAMAH_TGRPXLi6u7saYWpWQCBhaVH5vl7vhLhbsIIaEDiJ92DsH3HAcgA0T3Fb9iQ-VcChdkDcspCF5WTnXpJBxP4Th1XmxlZuYq_yxXR0D"

# full_red_list = []
# length_list = []

# browser = initialize_chrome_driver()
# country_pages = [argentina_red, australia_red, austria_red, chile_red, france_red, germany_red, italy_red, portugal_red, spain_red, usa_red]
# for page in country_pages:
#     country_list = scroll_and_load(browser, page, timeout=0.5, scrolls=200)
#     length_list.append(len(country_list))
#     full_red_list.append(country_list)
#     print("{} finished and gained {} links".format(page, len(country_list)))

In [27]:
# argentina_white = "https://www.vivino.com/explore?e=eJwNi7sKgDAMAP8mcxXXLC6u7iISYy0F20oaX39vlrvhuCTYQIoZHSR6sXMO-MOhBzaMcFoNO94k0SsdUFYU0phDXbhcWaHg5ivDo9OMra3VTPIDsT4c7w=="
# australia_white = "https://www.vivino.com/explore?e=eJwNizsOgCAQBW_zajS229jY2htj1lUJiYCBxc_tpZkpJuMTNfAukIHnlzpjIB8NPaRixFWrPejm5HblE3GlxOqCzYvEEhSRtj0LHp1mauuaq7n8sUEc8g=="
# austria_white = "https://www.vivino.com/explore?e=eJwNizsOgCAQBW_zajS229jY2htj1hUJiYCB9Xd7aWaKyYRMDYKPZBD4pc4YyEdDD6kYcdbqdro5e6t8IK2UWX10ZZF0RUWizRbBo9NMbV1LNesPsUAc8Q=="
# chile_white = "https://www.vivino.com/explore?e=eJwNizsKgDAQBW_z6ii229jY2ovIumoI5CPJ-ru9aWaKYUKmBsFFMgj8UmcM5KOhh1SMOGu1B92c3a7skVbKrC7aski6oiLRthfBo9NMbV1LtfgfsTwc6w=="
# france_white = "https://www.vivino.com/explore?e=eJwNi7sKgDAMAP8mcxXXLC6u7iISoy0F20oaX39vlrvhuCTYQIoZHSR6sXMO-MOhBzaMcFoNHm-SuCsdUFYU0phDXbhcWaHgtleGR6cZW1ur2csPsUgc9A=="
# germany_white = "https://www.vivino.com/explore?e=eJwNizsOgCAQBW_zajS229jY2htj1hUJiYCB9Xd7aWaKyYRMDYKPZBD4pc4YyEdDD6kYcdbqdro5e6t8IK2UWX10ZZF0RUWizRbBo9NMbV1L9WZ_sTcc5Q=="
# italy_white = "https://www.vivino.com/explore?e=eJwNi7sKgDAMAP8mcxXXLC6u7iISYy0B20obX39vlrvhuFiwgSgJHUR6sXMO-MOhBzaMcFoNO95UxCsdkFcspJJCXThfSSHj5ivDo9OMra3VLPoDsVAc-Q=="
# portugal_white = "https://www.vivino.com/explore?e=eJwNizsKgDAQBW-zdRTb19jY2ovIumoImA_J-ru9aWaKYXxGQ94FGPL8ojOG5MPQk1SMlGq1B27Oblc-Ka7IrC7Yski8glLEthehR6cZbV1LddIfsV4dAA=="
# spain_white = "https://www.vivino.com/explore?e=eJwNi7sKgDAMAP8mcxXXLC6u7iISYy0F20oTX39vlrvhuFSxgRQzOkj0Yucc8IdDD2wY4bQadrypRq90QFmxksYcZOFyZYWCmxeGR6cZW1vF7OUHsUcc9A=="
# usa_white = "https://www.vivino.com/explore?e=eJwNi7sKgDAMAP8mcxXXLC6u7iISo5aCbaVJffy9We6G42LBBmJI6CDSi51zwB8OPbBhhMuqP_CmEnalE_KKhTQkLwvnmhQybrswPDrN2Noq5io_sWcdBA=="

# full_white_list = []
# white_length_list = []

# browser = initialize_chrome_driver()
# country_white_pages = [argentina_white, australia_white, austria_white, chile_white, france_white, germany_white, italy_white, portugal_white, spain_white, usa_white]
# for page in country_white_pages:
#     country_list = scroll_and_load(browser, page, timeout=1, scrolls=150)
#     white_length_list.append(len(country_list))
#     full_white_list.append(country_list)
#     print("{} finished and gained {} links".format(page, len(country_list)))

Check the resulting sample size for red and white wines:

In [28]:
# records = 0
# for i in full_red_list:
#     records += len(i)
# print('our search gained {} red wine records'.format(records))

In [29]:
# records = 0
# for i in full_white_list:
#     records += len(i)
# print('our search gained {} white wine records'.format(records))

Save results to a file (for convenience, using [pickle library](https://docs.python.org/3/library/pickle.html))

In [30]:
# import pickle
# with open("popular_reds_sample", 'wb') as f:
#     pickle.dump(full_red_list, f)

In [31]:
# with open("popular_whites_sample", 'wb') as f:
#     pickle.dump(full_white_list, f)

### Parsing the properties of each wine from the list 

Trying to get interesting information per each wine, such as: 
- id:
    * wine name (this usualy includes also the year) (`class="wine"`)
- properties:
    * winery (`class="winery"`)
    * wine type (`class="wineLocationHeader__wineType--14nrC"`)
    * grapes (`class="wineFacts__fact--3BAsi"`) 
    * wine style (`class="wineFacts__fact--3BAsi"`) 
    * region (`class="anchor__anchor--3DOSm"`)
    * country (`class="wineLocationHeader__country--1RcW2"`)
- quality information:
    * number of reviews (`class="vivinoRatingWide__basedOn--s6y0t"`)
    * average price (`class="purchaseAvailabilityPPC__amount--2_4GT"`)
    * rating score (`class="vivinoRatingWide__averageValue--1zL_5"`)
    * 3 random community reviews (`class="reviewCard__reviewNote--fbIdd"`)
- taste structure:
    * there are 4 progress bars: light/bold, smooth/tannic, dry/sweet, soft/acidic 
    * each taste progressbar has the following class: `class="indicatorBar__progress--3aXLX"` with the following style properties identifying the bar location: `style="width: 15%; left: 85%;"` 
    * notes mention (`class="tasteNote__popularKeywords--1q7RG"`) 
    
Some of these elements are not visible until the page is scrolled down till the end. Therefore, just as with the wine list, we might need to scroll down, but this time without a pause. Instead, we will scroll until the specific element in the bottom becomes visible. This will ensure that all other elements are visible too. The bottom element chosen for this purposes has a class `addWidgetLink__addWidgetLink--aPZ_V`. 

In [32]:
# scroll until the end of a specific wine page 

# def scroll_until_the_end_of_wine_page(browser, page, page_end_class="addWidgetLink__addWidgetLink--aPZ_V"):
#     browser.get(page)
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     try:
#         WebDriverWait(browser, 50).until(EC.visibility_of_element_located((By.CLASS_NAME, page_end_class)))
#     except TimeoutException:
#         print("exception raised")

Once the whole wine page is loaded, we will need to check existence of each item that we need for analysis. 

To parse the data, important to note that each of page items has either a unique class name (in this case a single element can be extracted using a method `browser.find_element_by_class_name()`), or a generic class name that applies to more than one element (and therefore the list of elements can be extracted using a method `browser.find_elements_by_class_name()` and then indexed to get the desired data). 

Generally, we are interested in the text property of a given web element, however, in specific cases (eg. for the wine taste structure that is represented as a slidebar on vivino web page) we are interested in web element attributes instead.  

Some of the required information may be missing on a wine page, in such cases no exception should be raised. Instead, the program should consider `np.nan` as the correct data input. 

In order to consider the above specifics, we introduce a function `extract_data_with_exceptions` that takes several optional boolean arguments (`multiple_elements` tells whether a given class has more than one elements on a page, `get_style` tells whether we need attribute of a web element instead of text)


In [33]:
# extract either a single element, or a list of multiple elements
# def extract_data_with_exceptions(browser, class_name, multiple_elements=False, get_style=False):
#     if multiple_elements:
#         try:
#             list_with_data = []
#             data_list = browser.find_elements_by_class_name(class_name)
#             for element in data_list:
#                 if get_style:
#                     list_with_data.append(element.get_attribute("style"))
#                 else:
#                     list_with_data.append(element.text)
#         except:
#             list_with_data = "Not available"
#         return list_with_data
#     else:
#         try:
#             if get_style:
#                 data = browser.find_element_by_class_name(class_name).get_attribute("style")
#             else:
#                 data = browser.find_element_by_class_name(class_name).text
#         except:
#             data = np.nan
#         return data

The data will be stored in a pandas DataFrame. Therefore, for convenience, it was chosen to extract individual wine data in the form of a Python dictionary, that will be appended to the target DataFrame one by one. 

In [34]:
# def extract_data_as_dict(browser):
#     data_dict = {}
#     # extract wine name - OK text
#     data_dict["wine_name"] = extract_data_with_exceptions(browser, "vintage") 
#     # extract winery - OK text
#     data_dict["winery"] = extract_data_with_exceptions(browser, "winery")
#     # extract wine type - OK text
#     data_dict["wine_type"] = extract_data_with_exceptions(browser, "wineLocationHeader__wineType--14nrC")
#     # extract grapes - OK text list element
#     facts_list = extract_data_with_exceptions(browser, "wineFacts__fact--3BAsi", multiple_elements=True)
#     data_dict["grapes"] = facts_list[1] if len(facts_list) > 1 else np.nan
#     # extract wine style - OK text list element
#     data_dict["wine_style"] = facts_list[3] if len(facts_list) > 3 else np.nan
#     # extract region - OK text 
#     data_dict["region"] = extract_data_with_exceptions(browser, "anchor__anchor--3DOSm")
#     # extract country - OK text
#     data_dict["country"] = extract_data_with_exceptions(browser, "wineLocationHeader__country--1RcW2")
#     # extract the number of reviews - OK text 
#     data_dict["reviews"] = extract_data_with_exceptions(browser, "vivinoRatingWide__basedOn--s6y0t")
#     # extract the average price - OK text
#     data_dict["price"] = extract_data_with_exceptions(browser, "purchaseAvailabilityPPC__amount--2_4GT")
#     # extract the rating score - OK text
#     data_dict["score"] = extract_data_with_exceptions(browser, "vivinoRatingWide__averageValue--1zL_5")
#     # extract 3 reviews - OK text list
#     reviews_list = extract_data_with_exceptions(browser, "reviewCard__reviewNote--fbIdd", multiple_elements=True)
#     data_dict["review_1"] = reviews_list[0] if len(reviews_list) > 0 else np.nan
#     data_dict["review_2"] = reviews_list[1] if len(reviews_list) > 1 else np.nan
#     data_dict["review_3"] = reviews_list[2] if len(reviews_list) > 2 else np.nan
#     # extract taste bar
#     taste_list = extract_data_with_exceptions(browser, "indicatorBar__progress--3aXLX", multiple_elements=True, get_style=True)
#     data_dict["taste_light_bold"] = taste_list[0] if len(taste_list) > 0 else np.nan
#     data_dict["taste_smooth_tannic"] = taste_list[1] if len(taste_list) > 1 else np.nan
#     data_dict["taste_dry_sweet"] = taste_list[2] if len(taste_list) > 2 else np.nan
#     data_dict["taste_soft_acidic"] = taste_list[3] if len(taste_list) > 3 else np.nan
#     # extract keywords
#     keyword_list = extract_data_with_exceptions(browser, "tasteNote__popularKeywords--1q7RG", multiple_elements=True)
#     data_dict["keywords_1"] = keyword_list[0] if len(keyword_list) > 0 else np.nan
#     data_dict["keywords_2"] = keyword_list[1] if len(keyword_list) > 1 else np.nan
#     data_dict["keywords_3"] = keyword_list[2] if len(keyword_list) > 2 else np.nan
#     return data_dict

In [35]:
# df_25_wines = pd.DataFrame()

# for page in wine_pages_list:
#     scroll_until_the_end_of_wine_page(browser, page, page_end_class="addWidgetLink__addWidgetLink--aPZ_V")
#     cur_wine_data = extract_data_as_dict(browser)
#     df_25_wines = df_25_wines.append(cur_wine_data, ignore_index=True)

In [36]:
# df_25_wines.head()

Now when we know that the web scraper works correctly, we might want to apply it to a bigger subset of wines at the same time (eg. about 20k red wines that were saved in the script above).

However, once the script was run, it managed to load data about app. 340 wines before it triggered the bulk request limits on vivino.com.

The following message was thrown: 
*Your IP address (185.192.69.14) has been temporarily blocked for exceeding bulk request limits. If you believe this was done in error or you have legitimate needs to access our pages and data above and beyond these limits please contact admin@vivino.com with the subject 'Requests Blocked' and we'll try and resolve the issue.*

Still, for documentation purposes, the below code is kept here, and might be un-commented if needed. 

In [37]:
# wine_df_new = pd.DataFrame()

# browser = initialize_chrome_driver()

# start_time = time.time()
# for page in list_with_scroll:
#     scroll_until_the_end_of_wine_page(browser, page, page_end_class="addWidgetLink__addWidgetLink--aPZ_V")
#     cur_wine_data = extract_data_as_dict(browser)
#     wine_df_new = wine_df_new.append(cur_wine_data, ignore_index=True)
# end_time = time.time()

# print(wine_df_new)
# print("time elapsed: " + str(end_time - start_time) + " s.")

# wine_df_new.to_csv('red_wine_highest_score.csv')

### 3. Using Vivino API

##### Wine data

During a more detailed review of vivino request/response pairs, it was discovered that vivino has its own API not shown to the users, it returns a structured JSON string. The JavaString request to explore page looks as follows:
https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&page=4&price_range_max=400&price_range_min=0&wine_type_ids[]=1

From the format of the request sent to vivino servers, it can be seen that the following arguments are passed to the request to explore page: 
* `country_code`
* `currency_code`
* `grape_filter`
* `min_rating`
* `order_by`
* `order`
* `page`
* `price_range_max`
* `price_range_min`
* `wine_type_ids[]`

Here, in order to ensure that all wines are included to the request, we might expand the search criteria by setting the wide price range. Further, since the data is loaded by pages, we can iterate by page number until the whole database is loaded.

But first, we check which data can be retrieved (wine data & reviews) using this method.

We will simply use `requests` library and pass the headers specifying that json is expected in return (otherwise, the request might throw IP blocking for bulk request)

In [38]:
# test_page = "https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&page=4&price_range_max=400&price_range_min=0&wine_type_ids[]=1"

# headers_api = {
#     'Accept': 'application/json',
#     'Content-Type': 'application/json',
#     'User-Agent': 'python/requests',
# }

# response = requests.get(test_page, headers=headers_api)
# test_result = response.content

Since vivino.com server returns a JSON string, we might want to convert it to a Python-readable format. We will use `json` library for this task that converts string into a Python dictionary. 

In [39]:
import json 
# json_obj = json.loads(test_result)

In [40]:
# test_df = pd.DataFrame(json_obj['explore_vintage']['matches'])
# test_df.shape

We can see that a single page request yields 25 wined. We need to 'unroll' the vintage column which is by itself a dictionary. The results are stored in the `new_test_df`.

In [41]:
# new_test_df = pd.DataFrame()
# for row in range(len(test_df)):
# #     print(row)
#     new_test_df = new_test_df.append(test_df['vintage'][row], ignore_index=True)

In [42]:
# new_test_df.head(2)

We can see that a bunch of useful data is stored in 'wine' and 'statistics' columns that might be unrolled further. 

In [43]:
# def extract_wine_info(df):
#     for stat_field in ['ratings_count', 'ratings_average', 'labels_count']:
#         df[stat_field] = df['statistics'].apply(lambda x: x[stat_field]) 
#     df['wine_id'] = df['wine'].apply(lambda x: x['id'])
#     df['wine_name'] = df['wine'].apply(lambda x: x['name'])
#     df['type'] = df['wine'].apply(lambda x: x['type_id'])
#     df['vintage_type'] = df['wine'].apply(lambda x: x['vintage_type'])
#     df['country'] = df['wine'].apply(lambda x: x['style']['country']['name'])
#     df['region'] = df['wine'].apply(lambda x: x['region']['name'])
#     df['winery'] = df['wine'].apply(lambda x: x['winery']['name'])
#     for taste in ['acidity', 'fizziness', 'intensity', 'sweetness', 'tannin']:
#         df[taste] = df['wine'].apply(lambda x: x['taste']['structure'][taste])
#     for style_char in ['id', 'regional_name', 'varietal_name', 'body', 'body_description', \
#                   'acidity', 'acidity_description', 'grapes']:    
#         df['style_' + style_char] = df['wine'].apply(lambda x: x['style'][style_char])
#     df['flavor'] = df['wine'].apply(lambda x: x['taste']['flavor'])
#     return df

In [44]:
# test_df_extended = extract_wine_info(new_test_df)

In [45]:
# test_df_extended.columns

Once the necessary columns are extracted, we can drop some columns that are not required any longer

In [46]:
# test_df_extended.drop(['grapes', 'image', 'seo_name', 'statistics', 'top_list_rankings', 'wine']\
#                               , inplace=True, axis=1)

In [47]:
# test_df_extended.head(2)

#### Reviews data

From the looks of the API request we can see that it takes the following arguments:
* wine ID
* year
* number of reviews per page

We test sending request to reviews API, and it also results in a JSON string with requested information.

In [48]:
# test_reviews_page = "https://www.vivino.com/api/wines/83496/reviews?year=2016&per_page=15"
# response = requests.get(test_reviews_page, headers=headers_api)
# reviews_test_result = response.content

In [49]:
# json_obj = json.loads(reviews_test_result)
# reviews_test_df = pd.DataFrame(json_obj['reviews'])

In [50]:
# reviews_test_df.head(2)

According to the email from Birkir Barkarson (the CTO of vivino), the requests should be limited to 1000 per 10 minute window in order to avoid major interrution to their servers and resulting IP blockerage. 

Therefore, the `ratelimiter` is set accordingly. It will respect the limit of 1 request per second.

In [51]:
# all preprocessing steps to get from JSON string to a row in a DataFrame

def extract_json_to_df(df, json_str):
    json_obj = json.loads(json_str)
    for wine in json_obj['explore_vintage']['matches']:
#         print(type(wine))
        df = df.append(wine['vintage'], ignore_index=True)
    return df
        
# my_df = pd.DataFrame()
# my_df = extract_json_to_df(my_df, test_result)
# my_df.head(2)

In [52]:
from ratelimiter import RateLimiter

In [426]:
test_page = "https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&\
page=4&price_range_max=400&price_range_min=0&wine_type_ids[]=1"

headers_api = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'User-Agent': 'python/requests',
}

@RateLimiter(max_calls=1, period=1)
def get_wine_df(page, headers, df):
    response = requests.get(page, headers=headers)
    json_str = response.content
#     print(json_str)
    df = extract_json_to_df(df, json_str)
    return df

@RateLimiter(max_calls=1, period=1)
def get_wine_json(page, headers, matches):
    response = requests.get(page, headers=headers)
    json_str = response.content
    json_obj = json.loads(json_str)
    records_num = json_obj['explore_vintage']['records_matched']
#     print(records_num)
#     records.append(records_num)
    for match in json_obj['explore_vintage']['matches']:
        matches.append(match)
    return records_num, matches
#     print(time.time())


In [198]:
subset_red_df = pd.DataFrame()
subset_red_list = []
price_list = []

for i in range(1,60):
#     print(i)
    page = "https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&\
    order_by=ratings_average&order=desc&\
page={}&price_range_max=400&price_range_min=0&wine_type_ids[]=1".format(i)
    subset_red_df = get_wine_df(page, headers_api, subset_red_df)
    subset_red_list, price_list = get_wine_json(page, headers_api, subset_red_list, price_list)

In [235]:
import math
# print(math.ceil(4.2))

In [ ]:
1, iter_total + 1

In [429]:
full_df = pd.DataFrame()
full_match_list = []

page = 'https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&\
min_rating=1&order_by=ratings_average&order=desc&page={}&price_range_max=400&price_range_min=1'

records_total, _ = get_wine_json(page_new.format(1), headers_browser, full_match_list)
iter_total = math.ceil(records_total/100)
# # iter_total = 5
print(f"The program recognized app.{records_total} unique records and will run for app. {iter_total} iterations in total")

headers_browser = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
#     'User-Agent': 'python/requests',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',
#     'Cookie': 'first_time_visit=hS4ZPmom4T%2B8EVq37TqqHsTIJ79zRIt74blMfmiOLRrZSV%2FKl%2Btdl1LCq%2B%2Bpykg%2BUaEAQ18kFslfd1ZcYZHTz1Nn4SpvM0JrFg%3D%3D--qUx%2BZrxgAtAYUGLp--y6cajJQq6BpPk0nGFurazQ%3D%3D; eeny_meeny_test_checkout_login_v1=I%2FV0u9fALvvYvlXgTiDLvDJ7qTFF69POSotjBIVC8xggqMg6gSNg%2FRjGaTCnuNjzjXzMAC%2BdVUKS6DZa%2FI4D5Q%3D%3D; eeny_meeny_test_forced_merchant_filter_v1=nZoDv6SJ2FhC%2Fu%2Fc1WbAFi82MMN6VI5XI67vKEyf%2B2%2FIir%2BkNTg%2Bt8C2wvCm0BnuPyHcCYadh%2FhV9vl4%2BJqrIg%3D%3D; _ga=GA1.2.1380591255.1605016601; _gid=GA1.2.2136092280.1605016601; _fbp=fb.1.1605016601435.202579630; __auc=38ecb0ac175b271c5f4507ed27d; _hjTLDTest=1; _hjid=c884efdc-f057-45ed-b800-fff7a167c3fb; G_ENABLED_IDPS=google; client_cache_key=vBqztrSoTIt9R6jq3pVjVHtLshyIVnDvWq8H2cvl%2BAqMwnDtJstMI%2BRR5ueWvaSRlH%2FF%2F2sNGmzfjxNilGEN0NH7q89ohmdeRQ9Ig%2FtDdtSm%2Ftt3r0ZoBLOFGt%2FaVoP8OmSb7AcpOXPt--lFEik37GwoK%2FUF%2FO--wctHS4PJaaerWv%2BprunO2Q%3D%3D; __asc=e8c2fd05175bc1bec10ddf44c82; _hp2_ses_props.3503103446=%7B%22r%22%3A%22https%3A%2F%2Fwww.vivino.com%2Fvina-cobos-cobos-chanares-estate-malbec%2Fw%2F5354893%3Fyear%3D2015%22%2C%22ts%22%3A1605178747926%2C%22d%22%3A%22www.vivino.com%22%2C%22h%22%3A%22%2F%22%7D; _hjIncludedInSessionSample=1; _hjAbsoluteSessionInProgress=1; deal_merchant_context=TSOc--AFoK842%2BdYHklpcF--UyHGnHNX3xsq4Uhn1dI5ew%3D%3D; recently_viewed=JVgTKRd%2BGycVBS5P8Hz62dawNVVaKPAK4I6BaaDGqxA%2FXFBhvoW1385SOG%2Bk6EA%2FY0KAqW42jNHkzNlyMM%2Bhkfn0SYzRs3yW%2F4zeDi1u2Ncofmtk2kMHmMY4WM398kMQSoA4AB2j%2F1thGY%2B0nRqPW1KfUGQ0jruNn2IaZNF9wfslk3FLq8WJYCXRxa3br7%2BsM3K1I9C1Xn8Wtn3z0LbzAYij%2FFPboAq3isqHccNyetvmJ3bQF1sFU3TSnRKMnQ65ecaE2Qdj1NUceuMRS2o0cBQ3eYOfJVe%2FvfPBLwpLZXghG8Zl3NHNkQZikjskzsgKitLLzB6iSDSxrPGnURSgbwxAiVE1qR4jH6fEm0sZk45CSnk5tx9tr7X%2FNrIlQA6QtWaFDWCzZFRPdItVXJHxFgGk20X6dgwJVH0H7ZvTcAhVS4avtxkGDkwGv4fQ--iCtLq32wCUs7%2BP03--ypJsiJz4UNbEAjx9%2By5cEQ%3D%3D; _gat_vivinoTracker=1; _hp2_id.3503103446=%7B%22userId%22%3A%221928037028880653%22%2C%22pageviewId%22%3A%22769895479899750%22%2C%22sessionId%22%3A%226509757678601116%22%2C%22identity%22%3Anull%2C%22trackerVersion%22%3A%224.0%22%7D; _ruby-web_session=PUGxmp0Xk1PUxi%2B1FlSeTVni78H5g3D2NYGVPGrTiEzcYkoLmf2nALUjPXcT%2BCYSVROKA6bAocqcnGMyn8H28qHOhVw3RfihbeMkLbPte%2F2n2agXFmLMlae%2BKoHv8yv%2FsoJLCD2CUDmX3KKJ3msmUAhZYpN8O8IlXV7NmXy7SNlv2vghGR%2BfeOaF2gDctlbCzgnBlALy8vmUvwEmgSsGVGmmhN%2BuA7ovguyI2ZP%2FfT0iP0BcfCMaMlU3Lh2FNH9lgai2bElDAI2JlgICiCjc4zDHRIsQ96tJzzSlzV5Gdbyi%2BZrgI2eXygAv0mTkhpaCCwGlQS1GAFUXcgydlPVuVEf4Qg3FOisH0NNV%2FUDGEjAdw8nVb6IoG3Wat7tyTHiXe%2Fig9jxQpn7m1fzjv64RurUowOI1ixNMDPFvUPJCtm9ERqfr6ETgzICRFyuWse%2FAlCOzk61mBaqL%2FrpcUqzi6qb6F6oIbvmDo0I4Y5%2B%2Bi4dasCeBZ0n6dMbhDrzCNdHGObQtdXJBb3OS3kEtq9GTBv3mO6rhiCWvQEIohWo18kY2v2vMyZTf%2FPBFhjTR5ygPwZyuomQa6vXhGx%2BvYY5diho%3D--cMUu%2BKCwGqlFP7Wi--GfFzQ8ZKqG5M101xJ0GxFg%3D%3D',
#     'Referer': 'https://www.vivino.com/explore?e=eJzLLbI11jNVy83MszVQy02ssDUxMFBLrrR1d1JLBhIBagW2hmrpabZliUWZqSWJOWr5SbZFiSWZeenF8YllqUWJ6alq-bYpqcXJauUl0bFAxWDKCADUkx0A',
#     'If-None-Match': 'W/"30be28a898a520515cb59cab425bd9a5"',
    'Set-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'X-Requested-With': 'XMLHttpRequest'
    
}


full_match_list = []

timepoint_start = time.time()

max_records_bunch = 0

for i in range(400):
# for i in range(81, 83):
    if i == 0:
        timepoint_iter = timepoint_start
    
    cur_price_records_num = records_num[i] 
    iterations_required = math.ceil(cur_price_records_num/100)
    max_records = iterations_required * 100
    max_records_bunch += max_records
    for it in range(1, iterations_required + 1):
        cur_page = f"""https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&
        grape_filter=varietal&min_rating=1&order_by=ratings_count&order=desc&page={it}&per_page=100&price_range_max={i+1}&price_range_min={i}"""

        full_df = get_wine_df(cur_page, headers_browser, full_df)
        _, full_match_list = get_wine_json(cur_page, headers_browser, full_match_list)

        if i % 10 == 0 and it == iterations_required:
            with open(f"backup_data/match_list_{i-1}", 'wb') as f:
                pickle.dump(full_match_list[-max_records_bunch:], f)
            max_records_bunch = 0
            time_elapsed_bunch = time.time() - timepoint_iter
            timepoint_iter = time.time()
            print(f"Bunch of records with price up to {i+1} uploaded and took {time_elapsed_bunch/60} minutes. Next, running prices from {i+1}...")

#     records, _ = get_wine_json(cur_page, headers_browser, full_match_list, records)
with open(f"backup_data/full_match_list", 'wb') as f:
    pickle.dump(full_match_list, f)
timepoint_end = time.time()
time_elapsed = timepoint_end - timepoint_start
print(f"Program finished and successfully uploaded {len(full_match_list)} wine records with prices(need to check duplicates). The program took app. {time_elapsed/60} minutes to run")

The program recognized app.55846 unique records and will run for app. 559 iterations in total
Bunch of records with price up to 11 uploaded and took 8.327962017059326 minutes. Next, running prices from 11...
Bunch of records with price up to 21 uploaded and took 23.20164774656296 minutes. Next, running prices from 21...
Bunch of records with price up to 31 uploaded and took 16.013602952162426 minutes. Next, running prices from 31...
Bunch of records with price up to 41 uploaded and took 11.71570899883906 minutes. Next, running prices from 41...
Bunch of records with price up to 51 uploaded and took 8.694287796815237 minutes. Next, running prices from 51...
Bunch of records with price up to 61 uploaded and took 6.063190297285716 minutes. Next, running prices from 61...
Bunch of records with price up to 71 uploaded and took 4.508887720108032 minutes. Next, running prices from 71...
Bunch of records with price up to 81 uploaded and took 3.6940738519032794 minutes. Next, running prices fro

In [ ]:
full_match_list

In [424]:
records_num = records

In [430]:
# sum(records_num)

54799

In [425]:
# with open(f"backup_data/records_num", 'wb') as f:
#     pickle.dump(records_num, f)

In [423]:
records[-10:]

[6, 2, 8, 4, 18, 3, 7, 4, 10, 1501]

In [421]:
min(records)

0

In [395]:
records_list_full = records

In [418]:
max(records[1:])

1830

In [393]:
records.index(max(records[1:]))

1

In [394]:
records[1]

41433

In [ ]:
# max per page = 100
# max pages = 80

In [431]:
len(full_df)

55838

In [432]:
full_df['id'].nunique()

55819

In [341]:
f"""https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&
grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&page={i}&
price_range_max=400&price_range_min=1&wine_type_ids[]=1&wine_type_ids[]=1&
wine_type_ids[]=2&wine_type_ids[]=3&wine_type_ids[]=4&wine_type_ids[]=7&wine_type_ids[]=24"""

'https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&\ngrape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&page=2240&\nprice_range_max=400&price_range_min=1&wine_type_ids[]=1&wine_type_ids[]=1&\nwine_type_ids[]=2&wine_type_ids[]=3&wine_type_ids[]=4&wine_type_ids[]=7&wine_type_ids[]=24'

In [322]:
full_df.shape

(56000, 10)

In [334]:
full_df.isnull().sum()

grapes               56000
has_valid_ratings        0
id                       0
image                    0
name                     0
seo_name                 0
statistics               0
wine                     0
year                     0
top_list_rankings    55842
dtype: int64

In [337]:
full_df['id'].value_counts()

157653484.0    2160
74515331.0     2160
151244828.0    2160
3210749.0      2160
23504873.0     2160
               ... 
156173348.0       1
2048421.0         1
14290545.0        1
1262033.0         1
150998208.0       1
Name: id, Length: 2028, dtype: int64

In [338]:
full_df[full_df['id'] == 157653484]

,grapes,has_valid_ratings,id,image,name,seo_name,statistics,wine,year,top_list_rankings
2007,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
2032,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
2057,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
2082,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
2107,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
...,...,...,...,...,...,...,...,...,...,...
55884,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
55909,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
55934,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN
55959,None,1.0,157653484.0,{'location': '//images.vivino.com/thumbs/TiDhp...,Domaine Mouton Condrieu Côte Bonnette 2018,mouton-pere-et-fils-condrieu-cote-bonnette-2018,"{'status': 'Normal', 'ratings_count': 40, 'rat...","{'id': 1532153, 'name': 'Condrieu Côte Bonnett...",2018,NaN


In [252]:
# timepoint_2 = time.time()
# time_elapsed = timepoint_2 - timepoint_1
# print(f"Program finished and successfully uploaded {len(full_match_list)} wine records with prices. The program took app. {time_elapsed/60} minutes to run")

Program finished and successfully uploaded 56025 wine records with prices. The program took app. 183.52663078308106 minutes to run


In [259]:
# with open(f"backup_data/full_match_list", 'wb') as f:
#     pickle.dump(full_match_list, f)

In [433]:
with open(f"backup_data/full_match_list", 'rb') as f:
    recovered_data = pickle.load(f)

In [262]:
# with open(f"backup_data/full_match_df", 'wb') as f:
#     pickle.dump(full_df, f)

In [263]:
# with open(f"backup_data/full_match_df", 'rb') as f:
#     recovered_df = pickle.load(f)

In [270]:
# recovered_data[-1]

In [434]:
len(recovered_data)

55839

In [439]:
recovered_data[0]['vintage']['id']

111604237

In [442]:
distinct_dict = {entry['vintage']['id']: entry for entry in recovered_data}
recovered_data_distinct = distinct_dict.values()
len(recovered_data_distinct)

55819

In [436]:
type(recovered_data)

list

In [435]:
len(set(recovered_data))

TypeError: unhashable type: 'dict'

In [253]:
records

56017

In [325]:
len(full_match_list)

56000

In [249]:
with open(f"backup_data/full_match_list_5", 'rb') as f:
    backup_check = pickle.load(f)
len(backup_check)

125

In [234]:
records

56011

In [237]:
# test_price_list

In [205]:
len(test_price_list)

25

In [206]:
len(test_vintage_list)

25

In [ ]:
# subset_red_list[0]

In [ ]:
# response = requests.get("https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&\
# page=1&price_range_max=400&price_range_min=0&wine_type_ids[]=1", headers=headers_api)
# json_str = response.content
# json_obj = json.loads(json_str)
# matches = json_obj['explore_vintage']['matches']

In [ ]:
# subset_red_list = []

# for i in range(1, 60):
#     page = "https://www.vivino.com/api/explore/explore?country_code=GB&currency_code=GBP&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&\
# page={}&price_range_max=400&price_range_min=0&wine_type_ids[]=1".format(i)
#     subset_red_list = get_wine_json(page, headers_api, subset_red_list)

In [ ]:
# matches[0]['vintage']['wine']['region']

In [55]:
import pickle
with open("subset_red_list", 'wb') as f:
    pickle.dump(subset_red_list, f)

In [56]:
# import pickle
with open("subset_red_list", 'rb') as f:
    subset_red_list_backup = pickle.load(f)

In [60]:
# subset_red_df.shape

In [57]:
subset_red_df.columns

Index(['grapes', 'has_valid_ratings', 'id', 'image', 'name', 'seo_name',
       'statistics', 'wine', 'year', 'top_list_rankings'],
      dtype='object')

In [58]:
subset_red_df.head()

,grapes,has_valid_ratings,id,image,name,seo_name,statistics,wine,year,top_list_rankings
0,None,1.0,155452769.0,{'location': '//images.vivino.com/labels/bDEH9...,Marion Raro Cabernet Sauvignon Selezione 2007,marion-raro-cabernet-sauvignon-selezione-2007,"{'status': 'Normal', 'ratings_count': 83, 'rat...","{'id': 5926791, 'name': 'Raro Cabernet Sauvign...",2007.0,NaN
1,None,1.0,142749808.0,{'location': '//images.vivino.com/thumbs/BFAcH...,Frank Family Patriarch 2012,frank-family-patriarch-rutherford-red-wine-2012,"{'status': 'Normal', 'ratings_count': 80, 'rat...","{'id': 4382344, 'name': 'Patriarch', 'seo_name...",2012.0,NaN
2,None,1.0,57646109.0,{'location': '//images.vivino.com/labels/V5JCH...,Chateau D Yguene 2001,chateau-d-yguene-red-wine-v-jdmvq-2001,"{'status': 'Normal', 'ratings_count': 74, 'rat...","{'id': 3474900, 'name': 'Chateau D Yguene', 's...",2001.0,NaN
3,None,1.0,3208609.0,{'location': '//images.vivino.com/thumbs/easjT...,Del Dotto The Beast Cabernet Sauvignon 2012,del-dotto-the-beast-cabernet-sauvignon-2012,"{'status': 'Normal', 'ratings_count': 69, 'rat...","{'id': 1403662, 'name': 'The Beast Cabernet Sa...",2012.0,NaN
4,None,1.0,126346084.0,{'location': '//images.vivino.com/thumbs/4BpJn...,Hundred Acre Few and Far Between Cabernet Sauv...,hundred-acre-few-and-far-between-cabernet-sauv...,"{'status': 'Normal', 'ratings_count': 60, 'rat...","{'id': 4110288, 'name': 'Few and Far Between C...",2013.0,NaN


In [59]:
subset_red_df.columns = ['vintage_' + str(col) for col in subset_red_df.columns]

In [60]:
subset_red_df.columns

Index(['vintage_grapes', 'vintage_has_valid_ratings', 'vintage_id',
       'vintage_image', 'vintage_name', 'vintage_seo_name',
       'vintage_statistics', 'vintage_wine', 'vintage_year',
       'vintage_top_list_rankings'],
      dtype='object')

In [61]:
subset_red_stat = pd.json_normalize(subset_red_df['vintage_statistics'])
subset_red_stat.head(5)

,status,ratings_count,ratings_average,labels_count
0,Normal,83,4.9,316
1,Normal,80,4.9,341
2,Normal,74,4.9,624
3,Normal,69,4.9,357
4,Normal,60,4.9,312


In [62]:
subset_red_wine = pd.json_normalize(subset_red_df['vintage_wine'])
subset_red_wine.head(5)

,id,name,seo_name,type_id,vintage_type,is_natural,has_valid_ratings,region.id,region.name,region.name_en,...,style.region.background_image.location,style.region.background_image.variations.large,style.region.background_image.variations.medium,region,winery,style,taste.structure,region.background_image,style.background_image,style.region.background_image
0,5926791,Raro Cabernet Sauvignon Selezione,raro-cabernet-sauvignon-selezione,1,0,False,True,460.0,Veneto,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4382344,Patriarch,patriarch-rutherford-red-wine,1,0,False,True,105.0,Rutherford,,...,//images.vivino.com/regions/backgrounds/cP8NlY...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3474900,Chateau D Yguene,chateau-d-yguene-red-wine-v-jdmvq,1,0,False,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1403662,The Beast Cabernet Sauvignon,the-beast-cabernet-sauvignon,1,0,False,True,105.0,Rutherford,,...,//images.vivino.com/regions/backgrounds/cP8NlY...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4110288,Few and Far Between Cabernet Sauvignon,few-and-far-between-cabernet-sauvignon,1,0,False,True,25.0,Napa Valley,,...,//images.vivino.com/regions/backgrounds/cP8NlY...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,//thumbs.vivino.com/region_backgrounds/cP8NlYV...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
subset_red_flattened = pd.concat([subset_red_df, subset_red_stat, subset_red_wine], axis=1)

In [64]:
subset_red_flattened.isnull().head(5)

,vintage_grapes,vintage_has_valid_ratings,vintage_id,vintage_image,vintage_name,vintage_seo_name,vintage_statistics,vintage_wine,vintage_year,vintage_top_list_rankings,...,style.region.background_image.location,style.region.background_image.variations.large,style.region.background_image.variations.medium,region,winery,style,taste.structure,region.background_image,style.background_image,style.region.background_image
0,True,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
1,True,False,False,False,False,False,False,False,False,True,...,False,False,False,True,True,True,True,True,True,True
2,True,False,False,False,False,False,False,False,False,True,...,True,True,True,True,True,True,True,True,True,True
3,True,False,False,False,False,False,False,False,False,True,...,False,False,False,True,True,True,True,True,True,True
4,True,False,False,False,False,False,False,False,False,True,...,False,False,False,True,True,True,True,True,True,True


In [65]:
subset_red_flattened.iloc[:, :50].columns

Index(['vintage_grapes', 'vintage_has_valid_ratings', 'vintage_id',
       'vintage_image', 'vintage_name', 'vintage_seo_name',
       'vintage_statistics', 'vintage_wine', 'vintage_year',
       'vintage_top_list_rankings', 'status', 'ratings_count',
       'ratings_average', 'labels_count', 'id', 'name', 'seo_name', 'type_id',
       'vintage_type', 'is_natural', 'has_valid_ratings', 'region.id',
       'region.name', 'region.name_en', 'region.seo_name',
       'region.country.code', 'region.country.name',
       'region.country.native_name', 'region.country.seo_name',
       'region.country.currency.code', 'region.country.currency.name',
       'region.country.currency.prefix', 'region.country.currency.suffix',
       'region.country.regions_count', 'region.country.users_count',
       'region.country.wines_count', 'region.country.wineries_count',
       'region.country.most_used_grapes', 'region.background_image.location',
       'region.background_image.variations.large',
       '

In [66]:
subset_red_flattened.iloc[:, 50:].columns

Index(['taste.structure.user_structure_count',
       'taste.structure.calculated_structure_count', 'taste.flavor',
       'statistics.status', 'statistics.ratings_count',
       'statistics.ratings_average', 'statistics.labels_count',
       'statistics.vintages_count', 'style.id', 'style.seo_name',
       'style.regional_name', 'style.varietal_name', 'style.name',
       'style.image', 'style.background_image.location',
       'style.background_image.variations.small', 'style.description',
       'style.blurb', 'style.interesting_facts', 'style.body',
       'style.body_description', 'style.acidity', 'style.acidity_description',
       'style.country.code', 'style.country.name', 'style.country.native_name',
       'style.country.seo_name', 'style.country.currency.code',
       'style.country.currency.name', 'style.country.currency.prefix',
       'style.country.currency.suffix', 'style.country.regions_count',
       'style.country.users_count', 'style.country.wines_count',
       'st

In [67]:
subset_red_flattened.shape

(1475, 117)

In [68]:
# import pickle
# with open("subset_red_flattened", 'wb') as f:
#     pickle.dump(subset_red_flattened, f)

In [69]:
# %%capture
# %load_ext sql
# %sql sqlite:///vivino.db

In [70]:
# %sql SELECT * FROM wine

In [71]:
# %sql CREATE TABLE test_table (code text, name text);

In [72]:
# data = [{"country_code": "UK", "name": "UK"}, {"country_code": "IT", "name": "Italy"}]

# for entry in data:
#     code = entry["country_code"]
#     name = entry["name"]
#     %sql INSERT INTO test_table(code, name) VALUES (:code, :name);

In [73]:
# %sql SELECT * FROM test_table

In [74]:
import sqlite3

In [75]:
# test_query = 'SELECT * FROM test_table'
# print(conn.execute(test_query).fetchall())

In [76]:
# subset_red_flattened['winery.id']

Insertion to the database should be done in the following order (in order to avoid problems with foreign keys):
* winery
* country
* type
* region
* style
* food
* facts
* style_food
* grape
* style_grape
* grape_country
* wine
* vintage
* toplist
* vintage_toplist
* keyword
* wine_keyword
* wine_flavor_group

In [77]:
# subset_red_flattened.shape

In [78]:
#insert wineries
# conn = sqlite3.connect('vivino.db')

# timepoint_1 = time.time()
# for index, entry in subset_red_flattened.iterrows():
#     if pd.isnull(entry['winery.id']):
#         continue
#     else:
# #         print(entry['winery.id'])
#         query = "INSERT INTO winery (id, name, seo_name, status) SELECT ?, ?, ?, ?\
#         WHERE NOT EXISTS (SELECT * FROM winery WHERE id = ?);"
#         args = (entry['winery.id'], entry['winery.name'], entry['winery.seo_name'], entry['winery.status'], entry['winery.id'])
#         conn.execute(query, args)
        
# timepoint_2 = time.time()
# print('Insertion complete and took {} s.'.format(timepoint_2 - timepoint_1))

# conn.commit()

# if subset_red_flattened['winery.id'].nunique() == conn.execute('SELECT COUNT(*) FROM winery').fetchall()[0][0]:
#     print('Number of unique records is accurate')
# else:
#     print('Something went wrong')

# conn.close()

In [79]:
#insert countries
# conn = sqlite3.connect('vivino.db')

# timepoint_1 = time.time()
# for index, entry in subset_red_flattened.iterrows():
#     if pd.isnull(entry['region.country.code']):
#         continue
#     else:
# #         print(entry['winery.id'])
#         query = "INSERT INTO country \
#         SELECT ?, ?, ?, ?, ?, ?, ?, ?, ? \
#         WHERE NOT EXISTS (SELECT * FROM country WHERE code = ?);"
#         args = (entry['region.country.code'], \
#                 entry['region.country.name'], \
#                 entry['region.country.native_name'], \
#                 entry['region.country.seo_name'], \
#                 entry['region.country.currency.code'],\
#                 entry['region.country.regions_count'],\
#                 entry['region.country.users_count'],\
#                 entry['region.country.wines_count'],\
#                 entry['region.country.wineries_count'],\
#                 entry['region.country.code']
#                )
#         conn.execute(query, args)
        
# timepoint_2 = time.time()
# print('Insertion complete and took {} s.'.format(timepoint_2 - timepoint_1))

# conn.commit()

# if subset_red_flattened['region.country.code'].nunique() == conn.execute('SELECT COUNT(*) FROM country').fetchall()[0][0]:
#     print('Number of unique records is accurate')
# else:
#     print('Something went wrong')

# conn.close()

In [80]:
# insert wine types manually
# conn = sqlite3.connect('vivino.db')
# conn.execute("INSERT INTO type VALUES (1, 'Red'), (2, 'White'), (3, 'Sparkling'), (4, 'Rose'), (7, 'Dessert'), (24, 'Fortified');")
# conn.commit()
# conn.close()

In [81]:
# conn = sqlite3.connect('vivino.db')
# print(conn.execute("SELECT rootpage FROM sqlite_master WHERE name = 'grape_country';").fetchall()[0][0])
# conn.close()

In [82]:
# conn = sqlite3.connect('vivino.db')
# print(conn.execute("SELECT rootpage FROM sqlite_master WHERE name = 'grape_country';").fetchall()[0][0])
# conn.close()

In [83]:
# subset_red_flattened['type_id'].value_counts()

In [84]:
# subset_red_flattened.iloc[5][['type_id']].isnull().all()

In [85]:
# print(", ".join('?' * 5))

In [86]:
# tuple_built = tuple([char for char in 'Hello'])
# print(tuple_built)

In [87]:
# print([1,2,3] + [4,5,6])

In [88]:
# args list - names of desired columns in the dataframe, primary key list (names of columns considered as primary keys, may be one or more), name of the table

In [272]:
# recovered_data[0]['vintage']

In [162]:
def get_value(match_entry, path0):
#     print(match_entry)
    path = path0.split('/')
    current_el = match_entry
#     print(current_el)
    for p in path:
        if current_el is None:
            break
        current_el = current_el.get(p)
    return current_el

# columns_df = ['vintage/wine/region/id', 'vintage/wine/region/name', 'vintage/wine/region/name_en', 'vintage/wine/region/seo_name','vintage/wine/region/country/code']
# match_entry = matches[0]
# values_entry = [get_value(match_entry, col) for col in columns_df]

In [90]:
# subset_red_list[0]

In [91]:
# any([True])

In [443]:
vintage_list = [record['vintage'] for record in recovered_data_distinct]

In [275]:
# vintage_list[0]

In [276]:
#insert anything

def extract_json_to_sql(matches_list, table, paths, pk_sql, conn):

    # pk_cols_df = [columns_df[i] for i in range(len(pk_sql))]
    # if len(pk_sql) == 1: # only one primary key
    #     conditional_statement = "WHERE {} = ?);".format(pk_sql[0])
    # elif len(pk_sql) == 2: # two primary keys
    #     conditional_statement = "WHERE {} = ? AND {} = ?);".format(pk_sql[0], pk_sql[1])
    # else:
    #     raise ValueException
    conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)

    timepoint_1 = time.time()
    
    pk_paths = [paths[i] for i in range(len(pk_sql))]
#     print(pk_paths)
    # for index, entry in df.iterrows():
    for entry in matches_list:
        
        if any([get_value(entry, path) is None for path in pk_paths]):
#         if entry[pk_cols_df].isnull().any():
            continue
        else:
            query = f"""
              INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
              WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
            """

    #         values_entry = [entry[col] for col in columns_df]
            values_entry = [get_value(entry, path) for path in paths]
#             print(get_value(entry, 'statistics/status'))
            args = tuple(values_entry + values_entry[:len(pk_sql)])
    #         print(args)
    #         print(query)
            conn.execute(query, args)
        
    timepoint_2 = time.time()
    print('Insertion complete and took {} s.'.format(timepoint_2 - timepoint_1))

    conn.commit()

    # print(df[pk_cols_df].nunique()[0])
    # print(conn.execute('SELECT COUNT(*) FROM {}'.format(table)).fetchall()[0][0])
    # if df[pk_cols_df].nunique()[0] == conn.execute('SELECT COUNT(*) FROM {}'.format(table)).fetchall()[0][0]:
    unique_keys = set()
    for entry in matches_list:
        unique_key = []
        for path in pk_paths:
            unique_key.append(get_value(entry, path))
        if None in unique_key:
            # do nothing
            pass
        else:
            unique_keys.add(tuple(unique_key))
    unique_pk_original = len(unique_keys)
    
#     [get_value(entry, path) for path in pk_paths]
#     unique_pk_original = len(set([tuple([get_value(entry, path) for path in pk_paths]) for entry in matches_list]))
#     print(set([tuple([get_value(entry, path) for path in pk_paths]) for entry in matches_list]))
#     print(unique_pk_original)
    
    if unique_pk_original == conn.execute('SELECT COUNT(*) FROM {}'.format(table)).fetchall()[0][0]:
        print('Number of unique records is accurate')
    else:
        print('Something went wrong')

In [463]:
#data that should be passed to the function:
# df = subset_red_flattened
table = 'region'
paths = ['wine/region/id', 'wine/region/name', 'wine/region/name_en', 'wine/region/seo_name','wine/region/country/code']
pk_sql = ['id']
matches_list = vintage_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 0.9713551998138428 s.
Number of unique records is accurate


In [280]:
paths = ['wine/region/id', 'wine/region/name', 'wine/region/name_en', 'wine/region/seo_name','wine/region/country/code']
pk_sql = ['id']
pk_paths = [paths[i] for i in range(len(pk_sql))]
pk_paths

['wine/region/id']

In [445]:
len(vintage_list)

55819

In [311]:
# vintage_list[0]

In [446]:
# unique_keys = set()
unique_id = []

for entry in vintage_list:
#     print(entry['id'])
    unique_id.append(entry['id'])
    
# unique_key
len(set(unique_id))

55819

In [460]:
full_df['wine'].apply(lambda x: x['region']).apply(lambda x: x['id'] if x is not None else 0).nunique()

1663

In [452]:
full_df['wine'].head(5)

0    {'id': 1105374, 'name': 'Alandra Tinto', 'seo_...
1    {'id': 1706071, 'name': 'Alentejano Monte das ...
2    {'id': 4269600, 'name': 'Vinea Tinto', 'seo_na...
3    {'id': 1200770, 'name': 'Lisboa Tinto', 'seo_n...
4    {'id': 4269602, 'name': 'Vinea Branco', 'seo_n...
Name: wine, dtype: object

In [296]:
# full_df.to_excel('backup_file.xls')

In [464]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM region').fetchall())
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

[(1662,)]


Insertion to the database should be done in the following order (in order to avoid problems with foreign keys):
* winery - done
* country - done
* type - done
* region - done
* style - done
* food - done
* facts - done
* style_food - done
* grape - done
* style_grape - done
* grape_country - done
* wine - done
* vintage - done
* toplist - done
* vintage_toplist - done
* keyword
* wine_keyword
* wine_flavor_group

In [98]:
subset_red_flattened.iloc[:, :50].columns

Index(['vintage_grapes', 'vintage_has_valid_ratings', 'vintage_id',
       'vintage_image', 'vintage_name', 'vintage_seo_name',
       'vintage_statistics', 'vintage_wine', 'vintage_year',
       'vintage_top_list_rankings', 'status', 'ratings_count',
       'ratings_average', 'labels_count', 'id', 'name', 'seo_name', 'type_id',
       'vintage_type', 'is_natural', 'has_valid_ratings', 'region.id',
       'region.name', 'region.name_en', 'region.seo_name',
       'region.country.code', 'region.country.name',
       'region.country.native_name', 'region.country.seo_name',
       'region.country.currency.code', 'region.country.currency.name',
       'region.country.currency.prefix', 'region.country.currency.suffix',
       'region.country.regions_count', 'region.country.users_count',
       'region.country.wines_count', 'region.country.wineries_count',
       'region.country.most_used_grapes', 'region.background_image.location',
       'region.background_image.variations.large',
       '

In [99]:
subset_red_flattened.iloc[:, 50:].columns

Index(['taste.structure.user_structure_count',
       'taste.structure.calculated_structure_count', 'taste.flavor',
       'statistics.status', 'statistics.ratings_count',
       'statistics.ratings_average', 'statistics.labels_count',
       'statistics.vintages_count', 'style.id', 'style.seo_name',
       'style.regional_name', 'style.varietal_name', 'style.name',
       'style.image', 'style.background_image.location',
       'style.background_image.variations.small', 'style.description',
       'style.blurb', 'style.interesting_facts', 'style.body',
       'style.body_description', 'style.acidity', 'style.acidity_description',
       'style.country.code', 'style.country.name', 'style.country.native_name',
       'style.country.seo_name', 'style.country.currency.code',
       'style.country.currency.name', 'style.country.currency.prefix',
       'style.country.currency.suffix', 'style.country.regions_count',
       'style.country.users_count', 'style.country.wines_count',
       'st

In [103]:
subset_red_flattened.iloc[0]['style.food']

[{'id': 4,
  'name': 'Beef',
  'background_image': {'location': '//images.vivino.com/backgrounds/foods/4_beef.png',
   'variations': {'small': '//images.vivino.com/backgrounds/foods/thumbs/4_beef_932x810.png'}},
  'seo_name': 'beef'},
 {'id': 5,
  'name': 'Pasta',
  'background_image': {'location': '//images.vivino.com/backgrounds/foods/5_pasta.png',
   'variations': {'small': '//images.vivino.com/backgrounds/foods/thumbs/5_pasta_932x810.png'}},
  'seo_name': None},
 {'id': 8,
  'name': 'Lamb',
  'background_image': {'location': '//images.vivino.com/backgrounds/foods/8_lamb.png',
   'variations': {'small': '//images.vivino.com/backgrounds/foods/thumbs/8_lamb_932x810.png'}},
  'seo_name': 'lamb'},
 {'id': 11,
  'name': 'Game (deer, venison)',
  'background_image': {'location': '//images.vivino.com/backgrounds/foods/11_venison.png',
   'variations': {'small': '//images.vivino.com/backgrounds/foods/thumbs/11_venison_932x810.png'}},
  'seo_name': 'game'}]

In [465]:
table = 'style'
paths = ['wine/style/id', 'wine/style/seo_name', 'wine/style/regional_name', 'wine/style/varietal_name','wine/style/name',\
        'wine/style/description', 'wine/style/blurb', 'wine/style/body', 'wine/style/body_description', 'wine/style/acidity',\
        'wine/style/acidity_description', 'wine/style/country/code', 'wine/style/wine_type_id']
pk_sql = ['id']
matches_list = vintage_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 2.5976459980010986 s.
Number of unique records is accurate


In [466]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM style').fetchall()[0][0])
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

283


In [467]:
full_df['wine'].apply(lambda x: x['style']).apply(lambda x: x['id'] if x is not None else 0).nunique()

284

In [470]:
table = 'food'
food_list = []
for entry in vintage_list:
    if get_value(entry, 'wine/style/food') is not None:
        for food_element in get_value(entry, 'wine/style/food'):
            food_list.append(food_element)
# print(food_list)
paths = ['id', 'name', 'seo_name']
pk_sql = ['id']
matches_list = food_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 3.128354072570801 s.
Number of unique records is accurate


In [469]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM food').fetchall()[0][0])
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

10


In [111]:
subset_red_flattened.iloc[0]['style.interesting_facts']

['Until the last 20 years, Barbera was the most widely planted red variety in all of Italy.',
 'Many varieties of wine from Northern Italy are now being planted in California.',
 'Italy produces more wine than any country in the world accounting for nearly 1/3 of global production.']

In [471]:
table = 'facts'
paths = ['wine/style/id', 'wine/style/interesting_facts']
pk_sql = ['style_id', 'fact']
db = 'vivino.db'

conn = sqlite3.connect(db)

facts = []
for entry in vintage_list:
    style_id = get_value(entry, 'wine/style/id')
    if get_value(entry, 'wine/style/interesting_facts') is not None:
        fact_list = get_value(entry, 'wine/style/interesting_facts')
        for fact in fact_list:
            facts.append(fact)
            conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)

            query = f"""
                  INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
                  WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
                """
            values_entry = [style_id, fact]
            args = tuple(values_entry + values_entry)
    #         print(args)
    #         print(query)
            conn.execute(query, args)
        
print(len(set(facts)))
conn.commit()
conn.close()

497


In [472]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(DISTINCT(fact)) FROM facts').fetchall())
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

[(497,)]


In [473]:
table = 'style_food'
paths = ['wine/style/id', 'wine/style/food']
pk_sql = ['style_id', 'food_id']
db = 'vivino.db'

conn = sqlite3.connect(db)
pairs = []

for entry in vintage_list:
    style_id = get_value(entry, 'wine/style/id')
    if get_value(entry, 'wine/style/food') is not None:
        food_list = get_value(entry, 'wine/style/food')
        for food in food_list:
            conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)
            style_food_pair = (style_id, food['id'])
            pairs.append(style_food_pair)
            query = f"""
                  INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
                  WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
                """
            values_entry = [style_id, food['id']]
            args = tuple(values_entry + values_entry)
    #         print(args)
    #         print(query)
            conn.execute(query, args)
        
print(len(set(pairs)))
conn.commit()
conn.close()

1063


In [474]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM style_food').fetchall())
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

[(1063,)]


In [130]:
subset_red_flattened.iloc[0]['style.grapes']

[{'id': 12,
  'name': 'Nebbiolo',
  'seo_name': 'nebbiolo',
  'has_detailed_info': True,
  'wines_count': 48240},
 {'id': 31,
  'name': 'Barbera',
  'seo_name': 'barbera',
  'has_detailed_info': True,
  'wines_count': 43392},
 {'id': 37,
  'name': 'Dolcetto',
  'seo_name': 'dolcetto',
  'has_detailed_info': False,
  'wines_count': 10131}]

In [475]:
table = 'grape'
grape_list = []
for entry in vintage_list:
    if get_value(entry, 'wine/style/grapes') is not None:
        for grape in get_value(entry, 'wine/style/grapes'):
            grape_list.append(grape)
# print(food_list)
paths = ['id', 'name', 'seo_name', 'has_detailed_info', 'wines_count']
pk_sql = ['id']
matches_list = grape_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 2.480923891067505 s.
Number of unique records is accurate


In [476]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM grape').fetchall()[0][0])
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

128


In [477]:
table = 'style_grape'
paths = ['wine/style/id', 'wine/style/grapes']
pk_sql = ['style_id', 'grape_id']
db = 'vivino.db'

conn = sqlite3.connect(db)
pairs = []

for entry in vintage_list:
    style_id = get_value(entry, 'wine/style/id')
    if get_value(entry, 'wine/style/grapes') is not None:
        grape_list = get_value(entry, 'wine/style/grapes')
        for grape in grape_list:
            conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)
            style_grape_pair = (style_id, grape['id'])
            pairs.append(style_grape_pair)
            query = f"""
                  INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
                  WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
                """
            values_entry = [style_id, grape['id']]
            args = tuple(values_entry + values_entry)
    #         print(args)
    #         print(query)
            conn.execute(query, args)
        
print(len(set(pairs)))
conn.commit()
conn.close()

551


In [478]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM style_grape').fetchall())
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

[(551,)]


In [479]:
table = 'grape_country'
paths = ['wine/style/grapes', 'wine/style/country/code']
pk_sql = ['grape_id', 'country_code']
db = 'vivino.db'

conn = sqlite3.connect(db)
pairs = []

for entry in vintage_list:
    country_code = get_value(entry, 'wine/style/country/code')
    if get_value(entry, 'wine/style/grapes') is not None:
        grape_list = get_value(entry, 'wine/style/grapes')
        for grape in grape_list:
            conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)
            grape_country_pair = (grape['id'], country_code)
            pairs.append(grape_country_pair)
            query = f"""
                  INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
                  WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
                """
            values_entry = [grape['id'], country_code]
            args = tuple(values_entry + values_entry)
    #         print(args)
    #         print(query)
            conn.execute(query, args)
        
print(len(set(pairs)))
conn.commit()
conn.close()

283


In [480]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM grape_country').fetchall())
# conn.execute('DELETE FROM region')
# conn.commit()
conn.close()

[(283,)]


In [485]:
table = 'wine'
paths = ['wine/id', 'wine/name', 'wine/seo_name', 'wine/type_id', 'wine/vintage_type', 'wine/is_natural', 'wine/region/id', \
        'wine/winery/id', 'wine/taste/structure/acidity', 'wine/taste/structure/fizziness', 'wine/taste/structure/intensity',\
        'wine/taste/structure/sweetness', 'wine/taste/structure/tannin', 'wine/taste/structure/user_structure_count',\
        'wine/taste/structure/calculated_structure_count', 'wine/style/id', 'wine/statistics/status', 'wine/statistics/ratings_count',\
        'wine/statistics/ratings_average', 'wine/statistics/labels_count', 'wine/statistics/vintages_count', 'wine/has_valid_ratings']
pk_sql = ['id']
matches_list = vintage_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 2.162302255630493 s.
Number of unique records is accurate


In [486]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM wine').fetchall())
# conn.execute('DELETE FROM wine')
# conn.commit()
conn.close()

[(29811,)]


In [488]:
table = 'vintage'
paths = ['id', 'seo_name', 'name', 'wine/id', 'year', 'has_valid_ratings', \
         'statistics/status', 'statistics/ratings_count', 'statistics/ratings_average', 'statistics/labels_count']
pk_sql = ['id']
matches_list = vintage_list
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 1.528074026107788 s.
Number of unique records is accurate


In [489]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM vintage').fetchall())
# conn.execute('DELETE FROM vintage')
# conn.commit()
conn.close()

[(55819,)]


In [493]:
table = 'toplist'
toplist_list = []
for entry in vintage_list:
    if get_value(entry, 'top_list_rankings') is not None:
        for top_list in get_value(entry, 'top_list_rankings'):
            toplist_list.append(top_list)
paths = ['top_list/id', 'top_list/location', 'top_list/name', 'top_list/seo_name', 'top_list/type', 'top_list/year']
pk_sql = ['id']
matches_list = toplist_list
# print(matches_list)
db = 'vivino.db'

conn = sqlite3.connect(db)
extract_json_to_sql(matches_list, table, paths, pk_sql, conn)
conn.close()

Insertion complete and took 0.055648088455200195 s.
Number of unique records is accurate


In [494]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM toplist').fetchall())
# conn.execute('DELETE FROM toplist')
# conn.commit()
conn.close()

[(533,)]


In [498]:
table = 'vintage_toplist'
paths = ['id', 'top_list_rankings/top_list/id', 'top_list_rankings/rank', 'top_list_rankings/previous_rank',\
'top_list_rankings/description']
pk_sql = ['vintage_id', 'toplist_id']
db = 'vivino.db'

conn = sqlite3.connect(db)
pairs = []

for entry in vintage_list:
    vintage_id = get_value(entry, 'id')
    if get_value(entry, 'top_list_rankings') is not None:
        toplist_list = get_value(entry, 'top_list_rankings')
        if toplist_list is not None: 
            for toplist in toplist_list:
                conditional_statement = "WHERE " + " AND ".join(["{} = ?"] * len(pk_sql)).format(*pk_sql)
#                 print(toplist)
                vintage_toplist_pair = (vintage_id, toplist['top_list']['id'])
                pairs.append(vintage_toplist_pair)
                query = f"""
                      INSERT INTO {table} SELECT {', '.join('?' * len(paths))}
                      WHERE NOT EXISTS (SELECT * FROM {table} {conditional_statement})
                    """
                values_entry = [vintage_id, toplist['top_list']['id'], toplist['rank'], toplist['previous_rank'], toplist['description']]
                args = tuple(values_entry + [vintage_id, toplist['top_list']['id']])
        #         print(args)
        #         print(query)
                conn.execute(query, args)
        
print(len(set(pairs)))
conn.commit()
conn.close()

1900


In [499]:
conn = sqlite3.connect('vivino.db')
print(conn.execute('SELECT COUNT(*) FROM vintage_toplist').fetchall())
# conn.execute('DELETE FROM vintage_toplist')
# conn.commit()
conn.close()

[(1900,)]


In [500]:
with open('subset_red_list', 'rb') as f:
    recovered_subset = pickle.load(f)

In [501]:
recovered_subset[0]

{'id': 155452769,
 'seo_name': 'marion-raro-cabernet-sauvignon-selezione-2007',
 'name': 'Marion Raro Cabernet Sauvignon Selezione 2007',
 'statistics': {'status': 'Normal',
  'ratings_count': 83,
  'ratings_average': 4.9,
  'labels_count': 316},
 'image': {'location': '//images.vivino.com/labels/bDEH99ARSpCwFbDf3bct3w.jpg',
  'variations': {'large': '//images.vivino.com/thumbs/bDEH99ARSpCwFbDf3bct3w_375x500.jpg',
   'medium': '//images.vivino.com/thumbs/bDEH99ARSpCwFbDf3bct3w_150x200.jpg',
   'medium_square': '//images.vivino.com/thumbs/bDEH99ARSpCwFbDf3bct3w_150x150.jpg',
   'small_square': '//images.vivino.com/thumbs/bDEH99ARSpCwFbDf3bct3w_80x80.jpg'}},
 'wine': {'id': 5926791,
  'name': 'Raro Cabernet Sauvignon Selezione',
  'seo_name': 'raro-cabernet-sauvignon-selezione',
  'type_id': 1,
  'vintage_type': 0,
  'is_natural': False,
  'region': {'id': 460,
   'name': 'Veneto',
   'name_en': '',
   'seo_name': 'veneto',
   'country': {'code': 'it',
    'name': 'Italy',
    'native_na

In [ ]:
# %%sql

# SELECT * FROM winery LIMIT 5

In [ ]:
# %%sql

# INSERT INTO winery (id, name, seo_name, status) VALUES (1, 'bla', 'bla', 'bla');

In [ ]:
# test_test = pd.json_normalize(json_obj['reviews'])

In [ ]:
# test_test.head()

### Eugene comments 

Идеи по гипотезам
* Чем меньше популяция, относительно которой мы хотим делать выводы, тем проще нам будет собрать репрезентативную выборку. Поэтому вместо того, чтобы пытаться сделать вывод относительно всего рынка вин по миру, проще (и реалестичнее) пытаться делать выводы о более локальных популяциях- по странам, континентам.
* Например, может быть интересно посравнивать вина америки и европы, как два основных континента-поставщика.
    * Есть ли разница по оценкам между сортами? (некоторые сорта растут лучше в одном регионе, некоторые в другом)
    * Есть ли разница по сочетаниям? (гипотеза, что в европе больше сыра, в америке мяска)
    * Раазница по описаниям (какие слова используют для описания)
    * Сделать что-нибудь вроде описания прототипичного американского и европейского вин (взять средние вкусы/сорта/оценки, вывести средние, найти самое близкое из существующих к этому среднему - у нас есть прототипичные вина континетов! Можно разбить еще по сортам/ белому-красному/ еще чему-нибудь)
* Отедьно, конечно, интересно посмотреть выборку дешевых и дорогих вин.
    * Возможно, это будет сложно, но у меня есть фантазия взять данные по температурам в разные годы в регионах, где делается вино, и покоррелировать температуру и оценку/цену на вина. Можно ли предсказать цену на вино в винодельне по температуре?
    * Посмотреть облако слов для дешевых-средниих-супердорогих вин. Гипотеза, что описания дорогих вин будут более пафосными)
    * Посравнивать описания хороших оценок и плохих оценок для дешевых-средних-супер дорогих вин. Гипотеза - в дорогих винах людей неустраивают другие штуки, в сравнении с дешевыми (например, я посмотрел, что часто единички к дорогим винам ставят с пометкой crooked, плохо хранилось. Может еще будут инсайты)

### Касательно реперезентативности выборки
В зависимости от того, что мы в итоге будем хотеть проверять, намн ужно будет нагенерить репрезентативную выборку относительно именно той популяции, которой мы исследуем. Условно, если мы сравниваем америку и европу, нам нужны репрезентативные выборки по этим двум континентам. Мне кажется, что можно ограничиться следующими парааметрами:
* Страна
* Регион
* Год изготовления
* Тип (белое/красное/розовое/пузырики)
* Сорта
* Цена 

Нам нужно посмотреть распределение вин по этим переменным, и постараться заиметь похожее соотношение в нашей выборке. У нас есть информация по распределению по этим факторам по пупуляции? (например, по континентаам, странам, или по миру).

### One more thing...
Все эти идеи родились у меня в голове, а я про вино знаю чуть больше, чем ничего) Мне кажется, что по-настоящему клевые и интересные гипотезы у нас могут родиться, если мы почитаем про винаа чуть больше. Мне кжется, это вообще важный этап такой работы. Как в науке - делаешь литературный обрзор, потом формируешь гипотезы, потом думаешь о том, какие даанные тебе нужны, потом проверяешь гипотезы) Возможно, это overkill, хотя если цель - поупражняться в аналитике, то предварительный анализ - это важная ее часть. Тогда мы сможем оформить проектик почти как настоящую статью - с интродакшеном, референсами и прочим)


В общем скажи, что думаешь. Можно созвониться и обсудить. Может ты про вино знаешь больше меня и у тебя будут более интересные гипотезы)